In [1]:
import pandas as pd      
import numpy as np 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
#pip list

In [3]:
#pip install streamlit==1.10.0

In [4]:
#pip install scikit-learn==1.0.2

In [5]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mse = mean_squared_error(actual, pred)
    score = r2_score(actual, pred)
    return print(" r2_score:", score, "\n","mae:", mae, "\n","mse:",mse, "\n","rmse:",rmse)

In [7]:
df=pd.read_csv("final_df.csv")
pd.set_option('display.max_columns',  None)
pd.options.display.float_format = '{:,.1f}'.format

In [8]:
df.head(1)

,make_model,hp_kW,km,age,price,Gearing_Type,Gears
0,Audi A1,66.0,"56,013.0",3.0,15770,Automatic,7.0


In [9]:
df.shape

(15495, 7)

In [10]:
df.isnull().sum().any()

False

In [11]:
df.columns

Index(['make_model', 'hp_kW', 'km', 'age', 'price', 'Gearing_Type', 'Gears'], dtype='object')

In [12]:
df.age.unique()

array([3., 2., 1., 0.])

In [13]:
df.hp_kW.min()

40.0

In [14]:
df.hp_kW.max()

239.0

In [15]:
df.km.nunique()

6647

In [16]:
df.km.max()

317000.0

In [17]:
df.km.min()

0.0

In [18]:
df.Gearing_Type.unique()

array(['Automatic', 'Manual', 'Semi-automatic'], dtype=object)

In [19]:
df.make_model.unique()

array(['Audi A1', 'Audi A3', 'Opel Astra', 'Opel Corsa', 'Opel Insignia',
       'Renault Clio', 'Renault Duster', 'Renault Espace'], dtype=object)

In [20]:
df.make_model.value_counts()

Audi A3           3043
Audi A1           2609
Opel Insignia     2484
Opel Astra        2391
Opel Corsa        2216
Renault Clio      1749
Renault Espace     969
Renault Duster      34
Name: make_model, dtype: int64

In [21]:
df[df.make_model=="Audi A2"]

,make_model,hp_kW,km,age,price,Gearing_Type,Gears


In [22]:
df.drop(index=[2614], inplace =True)

In [23]:
df.make_model.unique()

array(['Audi A1', 'Audi A3', 'Opel Astra', 'Opel Corsa', 'Opel Insignia',
       'Renault Clio', 'Renault Duster', 'Renault Espace'], dtype=object)

In [24]:
new_list=["age", "hp_kW", "km", "Gearing_Type", 'make_model']

In [25]:
X=df[new_list]
y=df['price']

In [26]:
X.sample(5)

,age,hp_kW,km,Gearing_Type,make_model
1600,0.0,70.0,"9,600.0",Automatic,Audi A1
2606,0.0,70.0,"1,647.4",Manual,Audi A1
4385,1.0,85.0,"26,342.0",Manual,Audi A3
3714,1.0,85.0,"15,000.0",Automatic,Audi A3
12669,0.0,103.0,"1,647.4",Manual,Opel Insignia


In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [28]:
from sklearn.compose import make_column_transformer

In [29]:
from sklearn.preprocessing import OrdinalEncoder

In [30]:
cat = X_train.select_dtypes("object").columns
cat

Index(['Gearing_Type', 'make_model'], dtype='object')

In [31]:
ord_enc = OrdinalEncoder()
column_trans = make_column_transformer((ord_enc, cat), remainder='passthrough') # remainder: # Keep other columns rather than cat in original form.
                                                                                # remainder: # if you want to scale rest of cat columns use "minmax" istead passthroug
    
    # Apply ord_enc to categoric variables and remain in original form rest of them.
    # ord_enc for DT based algorithms
    # For Linear reg, log reg vs. use one hot encoder 

In [32]:
X_train=column_trans.fit_transform(X_train)

In [33]:
X_test=column_trans.transform(X_test)

In [34]:
X_train

array([[0.0000e+00, 2.0000e+00, 1.0000e+00, 1.1000e+02, 2.4000e+04],
       [0.0000e+00, 1.0000e+00, 1.0000e+00, 8.5000e+01, 3.3781e+04],
       [1.0000e+00, 4.0000e+00, 1.0000e+00, 1.0000e+02, 2.9421e+04],
       ...,
       [0.0000e+00, 3.0000e+00, 0.0000e+00, 6.6000e+01, 2.5000e+03],
       [1.0000e+00, 4.0000e+00, 2.0000e+00, 8.8000e+01, 9.3726e+04],
       [0.0000e+00, 1.0000e+00, 3.0000e+00, 8.1000e+01, 4.3250e+04]])

In [35]:
pd.Series(X_train[1])

0        0.0
1        1.0
2        1.0
3       85.0
4   33,781.0
dtype: float64

In [36]:
from sklearn.ensemble import RandomForestRegressor
rf_model=RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [37]:
y_pred = rf_model.predict(X_test)

In [38]:
eval_metrics(y_test,y_pred)

 r2_score: 0.9454462050623916 
 mae: 993.0336739767027 
 mse: 2387770.355419744 
 rmse: 1545.2411965190884


In [52]:
import pickle
pickle.dump(rf_model, open('rf_model', 'wb'))

In [40]:
pickle.dump(column_trans, open('transformer', 'wb'))

**deployment**

In [41]:
model = pickle.load(open('rf_model', 'rb'))

In [42]:
transformer = pickle.load(open('transformer', 'rb'))

In [43]:
columns=list(X.columns)
columns

['age', 'hp_kW', 'km', 'Gearing_Type', 'make_model']

In [44]:
my_dict = {
    "age": 2,
    "hp_kW": 105,
    "km": 100000,
    "Gearing_Type":"Automatic",
    "make_model": 'Audi A3'
}

In [45]:
#df = pd.DataFrame([my_dict])
#df = pd.DataFrame(my_dict, index=["a"])

In [46]:
df = pd.DataFrame.from_dict([my_dict])

In [47]:
df.head()

,age,hp_kW,km,Gearing_Type,make_model
0,2,105,100000,Automatic,Audi A3


In [48]:
df2 = transformer.transform(df)

In [49]:
pd.Series(df2[0])

0         0.0
1         1.0
2         2.0
3       105.0
4   100,000.0
dtype: float64

In [50]:
prediction = model.predict(df2)
print(prediction)

[18872.2]


In [51]:
print("The estimated price of your car is €{}. ".format(int(prediction[0])))

The estimated price of your car is €18872. 
